In [3]:
from communicate import CableRobot, MotorState, ControllerState
import numpy as np
import matplotlib.pyplot as plt
import time

In [34]:
# Function for recording data
def collect_data(duration=5):
    """Collects log data from the cable robot.  Returns tuple of Nx4 arrays for lengths and vels"""
    with CableRobot(print_raw=True, write_timeout=None, initial_msg='d10,1', port='/dev/tty.usbmodem103568503') as robot:
        tstart = time.time()
        while True:
            robot.update()
            time.sleep(0.01)
            if (time.time() - tstart > duration):
                break
        robot.send('d10,100')
    motor_ls = np.array([[m.length for m in datum['motors']] for datum in robot.all_data])
    motor_ldots = np.array([[m.lengthdot for m in datum['motors']] for datum in robot.all_data])
    xy = np.array([[dat['controller'].cur_x, dat['controller'].cur_y] for dat in robot.all_data])
    xy_com = np.array([[dat['controller'].set_x, dat['controller'].set_y] for dat in robot.all_data])
    return motor_ls, motor_ldots, xy, xy_com
def collect_points(N):
    motor_ls = []
    motor_ldots = []
    xy = []
    xy_commanded = []
    for i in range(N):
        input(f'Press enter to record point {i}')
        a, b, c, d = collect_data(0.5)
        motor_ls.append(a[-1])
        motor_ldots.append(b[-1])
        xy.append(c[-1])
        xy_commanded.append(d[-1])
        print(f'point {i}: ', motor_ls[-1], motor_ldots[-1], xy[-1], xy_commanded[-1])
    return np.array(motor_ls), np.array(motor_ldots), np.array(xy), np.array(xy_commanded)


In [35]:
collect_points(1)

point 0:  [-2.1657 -2.6816  2.8135  1.9785] [2.2417 0.261  0.0618 0.7263] [4.0604 0.6231] [4.0691 0.6144]


(array([[-2.1657, -2.6816,  2.8135,  1.9785]]),
 array([[2.2417, 0.261 , 0.0618, 0.7263]]),
 array([[4.0604, 0.6231]]),
 array([[4.0691, 0.6144]]))

In [37]:
collect_points(1)

point 0:  [-2.1717 -2.6958  2.8027  1.9812] [2.2421 0.2014 0.0618 0.6383] [4.0618 0.6258] [4.0724 0.6136]


(array([[-2.1717, -2.6958,  2.8027,  1.9812]]),
 array([[2.2421, 0.2014, 0.0618, 0.6383]]),
 array([[4.0618, 0.6258]]),
 array([[4.0724, 0.6136]]))

In [40]:
# Collect 4 corners, in order: BR, TR, TL, BL
# For each of the 4 corners, put the brush in "painting" position and just barely touch the mullions with the brush.
USE_CORNERS = True
if True:
    # Collect some data
    ls4, ldots4, xy4, xy_com4 = collect_points(4)
    print(ls4)
    print(ldots4)
    print(xy4)
    # Save/Load data (in case of ipynb failure)
    np.savez('/tmp/4pts_data.npz', ls4=ls4, ldots4=ldots4, est4=xy4, com4=xy_com4)
    np.savez(f'/tmp/4pts_data_{time.time()}.npz', ls4=ls4, ldots4=ldots4, est4=xy4, com4=xy_com4)
else:
    with np.load('/tmp/4pts_data.npz') as f:
        ls4 = f['ls4']
        ldots4 = f['ldots4']
        xy4 = f['est4']
        xy_com4 = f['com4']

# print(ls4)
# print(ldots4)
# print(xy4)
xy4 = xy_com4

point 0:  [-2.1739 -2.7306  2.8019  1.979 ] [2.2423 0.3458 0.1199 0.7218] [4.0634 0.631 ] [4.0716 0.6258]
point 1:  [-0.1529 -8.4853  0.1055  3.1364] [0.9332 0.9246 0.7233 0.4624] [4.0287 2.3757] [4.031 2.381]
point 2:  [ 2.9988 -0.0316 -5.9821 -0.0947] [0.3594 0.5782 1.28   0.4869] [1.9066 2.3991] [1.9037 2.4021]
point 3:  [ 1.7559  3.7874 -1.983  -2.1479] [1.067  0.0591 0.3627 1.9662] [1.9123 0.6701] [1.9058 0.6634]
[[-2.1739 -2.7306  2.8019  1.979 ]
 [-0.1529 -8.4853  0.1055  3.1364]
 [ 2.9988 -0.0316 -5.9821 -0.0947]
 [ 1.7559  3.7874 -1.983  -2.1479]]
[[2.2423 0.3458 0.1199 0.7218]
 [0.9332 0.9246 0.7233 0.4624]
 [0.3594 0.5782 1.28   0.4869]
 [1.067  0.0591 0.3627 1.9662]]
[[4.0634 0.631 ]
 [4.0287 2.3757]
 [1.9066 2.3991]
 [1.9123 0.6701]]


In [41]:
PANE = 'CENTER_BOTTOM'

In [46]:
# 4 Corner Constants
# First define X1, X2, Y1, Y2 are the bounds of the pane relative to the bottom-left pulley.
if PANE == 'CENTER_BOTTOM':
    PULLEY_O = .037
    X1 = 1.887 - PULLEY_O
    X2 = 4.022 - PULLEY_O
    Y1 = 5.417 - 4.614
    Y2 = Y1 + 1.816
elif PANE == 'CENTER_TOP':
    raise NotImplementedError
elif PANE == 'LEFT_MID_HALF':
    raise NotImplementedError
elif PANE == 'RIGHT_MID_HALF':
    raise NotImplementedError

print(f'sanity check: pane width (84"=2.1336m): {X2 - X1 = }')
if 'CENTER' in PANE:
    print(f'              total width (5.833m): {X2 + X1 = }')

BRUSH_R = (3.1e-2 + 0.8e-2) / 2 # 3.1cm brush diameter, 0.8cm fuzz factor
baseTbrush = np.array([21.0e-2, 38.5e-2])       # brush in the base coordinate frame
baseTcdprrest = -np.array([0.5334, 0.381]) / 2  # cdpr has a carriagewidth/2 offset we need to correct for
XY4_ORIGIN = np.array([[X2 - BRUSH_R, Y1 + BRUSH_R],
                [X2 - BRUSH_R, Y2 - BRUSH_R],
                [X1 + BRUSH_R, Y2 - BRUSH_R],
                [X1 + BRUSH_R, Y1 + BRUSH_R]]) - baseTbrush
print('Corner Locs: (using bottom-left coordinate system)')
print(XY4_ORIGIN)

print('Expected Corner Locs: (using carriage-center coordinate system)')
XY4 = XY4_ORIGIN - baseTcdprrest
print(XY4)
print('60mm buffer limits: (using carriage-center coordinate system)')
XY60 = np.array([[X2 - 0.06, Y1 + 0.06],
                 [X2 - 0.06, Y2 - 0.06],
                 [X1 + 0.06, Y2 - 0.06],
                 [X1 + 0.06, Y1 + 0.06]]) - baseTbrush - baseTcdprrest
print(XY60)
s = f'xLl{XY60[2][0]};xLr{XY60[0][0]};xLd{XY60[0][1]};xLu{XY60[1][1]}'
print(s)
print(f'gs0;{s};gs1;{s};gs2;{s};gs0;{s};')

sanity check: pane width (84"=2.1336m): X2 - X1 = 2.1350000000000002
              total width (5.833m): X2 + X1 = 5.835000000000001
Corner Locs: (using bottom-left coordinate system)
[[3.7555 0.4375]
 [3.7555 2.2145]
 [1.6595 2.2145]
 [1.6595 0.4375]]
Expected Corner Locs: (using carriage-center coordinate system)
[[4.0222 0.628 ]
 [4.0222 2.405 ]
 [1.9262 2.405 ]
 [1.9262 0.628 ]]
60mm buffer limits: (using carriage-center coordinate system)
[[3.9817 0.6685]
 [3.9817 2.3645]
 [1.9667 2.3645]
 [1.9667 0.6685]]
xLl1.9667000000000001;xLr3.9817000000000005;xLd0.6685;xLu2.3644999999999996
gs0;xLl1.9667000000000001;xLr3.9817000000000005;xLd0.6685;xLu2.3644999999999996;gs1;xLl1.9667000000000001;xLr3.9817000000000005;xLd0.6685;xLu2.3644999999999996;gs2;xLl1.9667000000000001;xLr3.9817000000000005;xLd0.6685;xLu2.3644999999999996;gs0;xLl1.9667000000000001;xLr3.9817000000000005;xLd0.6685;xLu2.3644999999999996;


In [43]:
# Compute an xy homography correction using the 4 corners
def compute_H(act, est):
    # act is 2x4, est is 2x4
    A = np.zeros((8, 9))
    for i in range(4):
        A[2*i, 0:3] = act[:, i]
        A[2*i, 6:9] = -est[0, i] * act[:, i]
        A[2*i+1, 3:6] = act[:, i]
        A[2*i+1, 6:9] = -est[1, i] * act[:, i]
    _, _, V = np.linalg.svd(A)
    H = V[-1, :].reshape(3, 3)
    return H

def make_homogeneous(x):
    return np.vstack((x, np.ones(x.shape[1])))

In [44]:
H = compute_H(make_homogeneous(XY4.T), make_homogeneous(xy4.T))
print(H)

# Sanity check
def apply_H(good):
    pred = H @ make_homogeneous(good.T)
    pred /= pred[-1, :]
    return pred[:-1, :].T
print('applying homography to expected corners: ', apply_H(XY4))
print('actual measure corners:                  ', xy4)

[[ 0.58137413  0.01031624 -0.03520595]
 [-0.01184055  0.57405472  0.04211308]
 [-0.00250894  0.00577445  0.57372997]]
applying homography to expected corners:  [[4.0716 0.6258]
 [4.031  2.381 ]
 [1.9037 2.4021]
 [1.9058 0.6634]]
actual measure corners:                   [[4.0716 0.6258]
 [4.031  2.381 ]
 [1.9037 2.4021]
 [1.9058 0.6634]]


In [45]:
# Here are the limits we should use post-homography
print('60mm buffer limits:')
XY60_cdpr_coords = apply_H(XY60)
print(XY60_cdpr_coords)
xmin, xmax = min(XY60_cdpr_coords[:, 0]), max(XY60_cdpr_coords[:, 0])
ymin, ymax = min(XY60_cdpr_coords[:, 1]), max(XY60_cdpr_coords[:, 1])
s = f'xLl{xmin};xLr{xmax};xLd{ymin};xLu{ymax}'
print(s)
print(f'gs0;{s};gs1;{s};gs2;{s};gs0;{s};')
print('The following should be typed into gcode webpage:')
print('H:')
print(H.tolist())
print('Expected Mural Translation:')
print(np.min(XY60, axis=0))
print('Expected Mural Dimensions (with the given scale factor, this should be the total width & height):')
print(np.max(XY60, axis=0) - np.min(XY60, axis=0))
# print((np.max(XY60, axis=0) - np.min(XY60, axis=0)) + 0.12)

60mm buffer limits:
[[4.0284468  0.66723552]
 [3.99042051 2.34211012]
 [1.9445044  2.36274394]
 [1.94720604 0.70300832]]
xLl1.9445043952568957;xLr4.02844679566135;xLd0.6672355217959156;xLu2.362743944133787
gs0;xLl1.9445043952568957;xLr4.02844679566135;xLd0.6672355217959156;xLu2.362743944133787;gs1;xLl1.9445043952568957;xLr4.02844679566135;xLd0.6672355217959156;xLu2.362743944133787;gs2;xLl1.9445043952568957;xLr4.02844679566135;xLd0.6672355217959156;xLu2.362743944133787;gs0;xLl1.9445043952568957;xLr4.02844679566135;xLd0.6672355217959156;xLu2.362743944133787;
The following should be typed into gcode webpage:
H:
[[0.5813741277729948, 0.01031623526040343, -0.03520594949787151], [-0.011840554749783615, 0.5740547151937571, 0.04211307662885223], [-0.0025089375018091764, 0.005774452266465378, 0.5737299668866128]]
Expected Mural Translation:
[1.9667 0.6685]
Expected Mural Dimensions (with the given scale factor, this should be the total width & height):
[2.015 1.696]


In [16]:
xy4

array([[4.0123, 0.671 ],
       [4.0105, 2.435 ],
       [1.918 , 2.4148],
       [1.9082, 0.656 ]])

In [ ]:
plt.plot(*XY60.T, 'r*')
plt.plot(*XY60_cdpr_coords.T, 'k*')

In [ ]:
# To test the gcode homography, for the first
X, Y = 1355.851/1e3, 12.000/1e3
print('Original: ', X, Y)
X, Y = (X * 0.94379391 + 1.9667), (Y *   + 0.6685)
print(X, Y)
print('To be sent to the robot:')
apply_H(np.array([[X, Y]]).reshape(1, 2))